In [1]:
# task2.py を逐次実行してみて中身を把握する

import sys
import os

import numpy as np
import argparse
import textwrap


import librosa
import librosa.display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1 import make_axes_locatable

sys.path.append('/store/projects/ml/mathG/DCASE2017/baseline/DCASE2017-baseline-system/')

from dcase_framework.application_core import BinarySoundEventAppCore
from dcase_framework.parameters import ParameterContainer
from dcase_framework.utils import *
from dcase_framework.features import FeatureContainer, FeatureRepository, FeatureExtractor, FeatureNormalizer, \
    FeatureStacker, FeatureAggregator, FeatureMasker


In [2]:
project_base=os.path.dirname(os.path.realpath('../../baseline/DCASE2017-baseline-system/applications/system/'))


params = ParameterContainer(
    project_base=project_base,
    path_structure={
        'feature_extractor': [
            'dataset',
            'feature_extractor.parameters.*'
        ],
        'feature_normalizer': [
            'dataset',
            'feature_extractor.parameters.*'
        ],
        'learner': [
            'dataset',
            'feature_extractor',
            'feature_stacker',
            'feature_normalizer',
            'feature_aggregator',
            'learner'
        ],
        'recognizer': [
            'dataset',
            'feature_extractor',
            'feature_stacker',
            'feature_normalizer',
            'feature_aggregator',
            'learner',
            'recognizer'
        ],
    }
)

params

{}

In [3]:
params.load(filename=project_base + '/parameters/task2.defaults.yaml')
params['active_set'] = 'dcase2017_gpu'
params.process() # 実行用のパラメータ構造に変える？


setup_logging(parameter_container=params['logging'])
#app.show_parameters()

/store/projects/ml/mathG/DCASE2017/baseline/DCASE2017-baseline-system/dcase_framework/files.py:379: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  dict.update(self, yaml.load(infile))


In [4]:
class Task2AppCore(BinarySoundEventAppCore):
    pass

app = Task2AppCore(
    name='DCASE 2017::Detection of rare sound events / Baseline System',
    params=params,
    system_desc=params.get('description'),
    system_parameter_set_id=params.get('active_set'),
    setup_label='Development setup',
    log_system_progress=params.get_path('general.log_system_progress'),
    show_progress_in_console=params.get_path('general.print_system_progress'),
    use_ascii_progress_bar=params.get_path('general.use_ascii_progress_bar')
)

# app.initialize()

[I] DCASE 2017::Detection of rare sound events / Baseline System
[I] 


In [5]:
app.feature_normalization()

[I] Feature normalizer
[I] ==================================================
[I] 


           Feature method :   0%|          | 0/1 [00:00<?, ?it/s]
                                                                 
           Feature method :   0%|          | 0/1 [00:00<?, ?it/s]
                                                                 
           Feature method :   0%|          | 0/1 [00:00<?, ?it/s]
                                                                 

[I]   DONE     [0:00:00.020553 ] 
[I] 


In [17]:
from tqdm import tqdm

fold_progress = tqdm(app._get_active_folds(),
                     desc='           {0:<15s}'.format('Fold '),
                     file=sys.stdout,
                     leave=False,
                     miniters=1,
                     disable=app.disable_progress_bar,
                     ascii=app.use_ascii_progress_bar)


for fold in fold_progress:
    if app.log_system_progress:
        app.logger.info('  {title:<15s} [{fold:d}/{total:d}]'.format(
            title='Fold',
            fold=fold,
            total=len(fold_progress))
        )

    event_labels = app.dataset.event_labels
    # Select only active events
    if app.params.get_path('feature_normalizer.active_events'):
        event_labels = list(
            set(event_labels).intersection(
                app.params.get_path('feature_normalizer.active_events')
            )
        )

    event_labels = ['babycry']
    for event_label in event_labels:
        current_normalizer_files = app._get_feature_normalizer_filename(
            fold=fold,
            path=app.params.get_path('path.feature_normalizer'),
            event_label=event_label
        )

        method_progress = tqdm(current_normalizer_files,
                               desc='           {0: >15s}'.format('Feature method '),
                               file=sys.stdout,
                               leave=False,
                               miniters=1,
                               disable=app.disable_progress_bar,
                               ascii=app.use_ascii_progress_bar)

        for method in method_progress:
            current_normalizer_file = current_normalizer_files[method]
#             if not os.path.isfile(current_normalizer_file) or overwrite:

            normalizer = FeatureNormalizer()
            item_progress = tqdm(app.dataset.train(fold, event_label=event_label).file_list,
                                 desc="           {0: >15s}".format('Collect data '),
                                 file=sys.stdout,
                                 leave=False,
                                 bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} ',
                                 disable=app.disable_progress_bar,
                                 ascii=app.use_ascii_progress_bar
                                 )

            for item_id, audio_filename in enumerate(item_progress):
                feature_filename = app._get_feature_filename(
                    audio_file=audio_filename,
                    path=app.params.get_path('path.feature_extractor', {})[method]
                )

                if app.log_system_progress:
                    app.logger.info(
                        '  {title:<15s} [{item_id:d}/{total:d}] {file:<30s}'.format(
                            title='Item',
                            item_id=item_id,
                            total=len(item_progress),
                            file=os.path.split(feature_filename)[-1])
                        )

                if os.path.isfile(feature_filename):
                    feature_stats = FeatureContainer(filename=feature_filename)
                else:
                    message = '{name}: Features not found [{file}]'.format(
                        name=app.__class__.__name__,
                        file=audio_filename
                    )

                    app.logger.exception(message)
                    raise IOError(message)

                # Accumulate statistics
                normalizer.accumulate(feature_stats)

            # Calculate normalization factors
            normalizer.finalize().save(filename=current_normalizer_file)



           Feature method :   0%|          | 0/1 [00:00<?, ?it/s]

             Collect data :   0%|          | 0/491 

             Collect data :   4%|▎         | 18/491 

             Collect data :   9%|▉         | 43/491 

             Collect data :  13%|█▎        | 66/491 

             Collect data :  16%|█▋        | 81/491 

             Collect data :  22%|██▏       | 106/491 

             Collect data :  26%|██▋       | 129/491 

             Collect data :  31%|███▏      | 154/491 

             Collect data :  36%|███▌      | 175/491 

             Collect data :  40%|███▉      | 195/491 

             Collect data :  44%|████▍     | 218/491 

             Collect data :  50%|████▉     | 244/491 

             Collect data :  55%|█████▍    | 269/491 

             Collect data :  59%|█████▉    | 292/491 

             Collect data :  64%|██████▍   | 316/491 

             Collect data :  69%|██████▉   | 341/491 

             Collect data :  75%|███████▍  | 367/491 

    

In [18]:
normalizer

{'N': [736991],
 'S1': [array([-3292190.8, -4026356.2, -4472570.5, -4745850.5, -4942060. ,
         -5099277. , -5200330. , -5255644. , -5305462.5, -5342108.5,
         -5377323. , -5439035.5, -5517021. , -5615883. , -5690855. ,
         -5754645.5, -5805114. , -5868702. , -5969166. , -6124990.5,
         -6303076. , -6380403. , -6374980.5, -6379427.5, -6430474. ,
         -6508635.5, -6636735. , -6854361.5, -7119328.5, -7410141. ,
         -7669264.5, -7719746. , -7822984. , -7983160.5, -8139666. ,
         -8236034.5, -8321340. , -8400945. , -8514879. , -8700120. ],
        dtype=float32)],
 'S2': [array([1.6203981e+07, 2.3420178e+07, 2.8299246e+07, 3.1692934e+07,
         3.4262720e+07, 3.6422732e+07, 3.7817892e+07, 3.8612344e+07,
         3.9364464e+07, 3.9930236e+07, 4.0444484e+07, 4.1296988e+07,
         4.2447904e+07, 4.3916120e+07, 4.5066848e+07, 4.6057500e+07,
         4.6855248e+07, 4.7878912e+07, 4.9524548e+07, 5.2071980e+07,
         5.5020008e+07, 5.6343892e+07, 5.6236436e

In [15]:
feature_stats

{'feat': [array([[ -5.002877 ,  -7.204636 ,  -7.887638 , ..., -11.675259 ,
          -11.7941675, -12.186553 ],
         [ -4.816158 ,  -6.169664 ,  -6.6384535, ..., -11.342943 ,
          -11.678137 , -12.109688 ],
         [ -4.704848 ,  -5.784425 ,  -6.270282 , ..., -11.4215975,
          -11.667576 , -12.06991  ],
         ...,
         [ -5.7667813,  -5.778595 ,  -6.9095564, ..., -11.737999 ,
          -11.727472 , -12.06486  ],
         [ -5.7177258,  -6.202236 ,  -6.744507 , ..., -11.686974 ,
          -11.807487 , -12.002305 ],
         [ -6.074368 ,  -6.2238717,  -7.357516 , ..., -11.659121 ,
          -11.758827 , -12.153561 ]], dtype=float32)],
 'stat': [{'mean': array([ -5.9018297,  -6.648721 ,  -7.010212 ,  -7.670514 ,  -7.8915057,
           -8.124262 ,  -8.314808 ,  -8.436348 ,  -8.459042 ,  -8.592465 ,
           -8.511891 ,  -8.298835 ,  -8.5117445,  -8.747477 ,  -8.881645 ,
           -8.932578 ,  -9.116269 ,  -9.237127 ,  -9.38967  ,  -9.622303 ,
           -9.959391